In [ ]:
import yfinance as yf

stock_symbol = '^GSPC'
stock = yf.Ticker(stock_symbol)
hist = stock.history(period="10y")
df = hist.drop(columns=['Dividends', 'Stock Splits'])
df.head()

In [ ]:
df['open'] = df['Open']
df['high'] = df['High']
df['low'] = df['Low']
df['close'] = df['Close']
df['volume'] = df['Volume']

In [ ]:
import pandas as pd
import pandas_ta as ta

def generate_patterns(df):
    patterns = df.ta.cdl_pattern(name=['doji', 'inside'])
    patterns = patterns.replace(0, -5000)
    return patterns
def generate_cycles(df):
    cycles = pd.DataFrame(index=df.index)
    cycles['ebsw'] =ta.ebsw(df['close'])
    cycles['atr'] = ta.atr(df['high'], df['low'], df['close'], length=7)
    z = ta.cdl_z(df['open'], df['high'], df['low'], df['close'])
    cycles['open_z'] = z['open_Z_30_1']
    cycles['high_z'] = z['high_Z_30_1']
    cycles['low_z'] =  z['low_Z_30_1']
    cycles['close_z'] = z['close_Z_30_1']
   
    
    return cycles
def generate_stats(df):
    t = 7
    stats = pd.DataFrame(index=df.index)
    
    stats['dev'] = ta.stdev(df['close'], length=t)
    stats['ema20'] = ta.ema(df['close'], length=20)
    stats['ema50'] = ta.ema(df['close'], length=50)
    # stats['ema100'] = ta.ema(df['close'], length=100)
    # stats['ema200'] = ta.ema(df['close'], length=200)
    stats['rsi6'] = ta.rsi(df['close'], length=6)
    stats['rsi12'] = ta.rsi(df['close'], length=12)
    stats['adx'] = ta.adx(df['high'], df['low'], df['close'], length=24)['ADX_24']
    # stats['sar'] = ta.sar(df['high'], df['low'])
    bbands = ta.bbands(df['close'], length=t, std=2.0)
    # print(bbands.columns)
    stats['b_upper'] = bbands['BBU_7_2.0']
    stats['b_middle'] = bbands['BBM_7_2.0']
    stats['b_lower'] = bbands['BBL_7_2.0']
    stats['percentage_difference'] = (df['high'] - df['low']) / df['low'] * 100
    stats['percentage_change'] = df['close'].pct_change()
    stats['min_price'] = df['close'].rolling(3).min()
    stats['max_price'] = df['close'].rolling(3).max()
    stats['next_day_close'] = df['close'].shift(-1)
    
    return stats

def generate_ind(df):
    t = 7
    stats = pd.DataFrame(index=df.index)

    stats['var'] = ta.variance(df['close'], length=t)
    a = ta.aroon(df['high'], df['low'], length=t)
    # stats['aroonD'], stats['aroonU'], stats['aroonSC'] = ta.aroon(df['high'], df['low'], length=t)
    stats['aroond'] = a['AROOND_7']
    stats['aroonu'] = a['AROONU_7']
    stats['aroonc'] = a['AROONOSC_7']
    # print(aroon.columns)
    # stats['aroon'] = aroon['Aroon Up'] - aroon['Aroon Down']
    stats['bop'] = ta.bop(df['open'], df['high'], df['low'], df['close'])
    stats['cci'] = ta.cci(df['high'], df['low'], df['close'], length=t)
    stats['mom'] = ta.mom(df['close'], length=t)
    stats['roc'] = ta.roc(df['close'], length=t) 
    # stats['rocp'] = ta.rocp(df['close'], length=t)
    stats['willr'] = ta.willr(df['high'], df['low'], df['close'], length=t)
    # stats = stats.astype(float)

    return stats

In [ ]:
stats = generate_stats(df)
patterns = generate_patterns(df)
cycles = generate_cycles(df)
cycles_cols = cycles.columns.to_list()
indicators = generate_ind(df)
df = pd.concat([df, patterns], axis=1)
df = pd.concat([df, cycles], axis=1)
df = pd.concat([df, stats], axis=1)
df = pd.concat([df, indicators], axis=1)
df = df.dropna()
# df = df.reset_index(drop=True)

In [ ]:
import joblib

sc = joblib.load('scaler2.pkl')
columns_norm = cycles.columns.to_list() + indicators.columns.to_list() 
df[columns_norm] = sc.transform(df[columns_norm])

In [ ]:
columns_norm

In [ ]:
df_test = df[df.index >= '2024-03-11']
df = df[df.index <= '2024-03-11']

In [ ]:
features = ['high','low','close'] +  stats.columns.to_list()[:-1] + columns_norm + patterns.columns.to_list()
X_lin = df[features].values.reshape(df[features].shape[0], 1, df[features].shape[1])
X_test = df_test[features].values.reshape(df_test[features].shape[0], 1, df_test[features].shape[1])
Y_lin = df['next_day_close'].values
Y_test = df_test['next_day_close'].values

In [ ]:
from tensorflow.python.keras.backend import set_session
import tensorflow.compat.v1 as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

In [ ]:
def r2_metric(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return (1 - SS_res / (SS_tot + tf.keras.backend.epsilon()))

In [ ]:
from keras.models import load_model
model = load_model('best_price4.keras', custom_objects={'r2_metric': r2_metric})

In [ ]:
from keras.callbacks import ModelCheckpoint

best_price = ModelCheckpoint('best_price4.keras', save_best_only=True, 
                            monitor='val_r2_metric', mode='max')

model.fit(X_lin,Y_lin, epochs=500, batch_size=32, verbose=1, shuffle=False, 
        validation_data=[X_test, Y_test], callbacks=[best_price])

In [ ]:
from keras.models import load_model
model = load_model('best_price4.keras',custom_objects={'r2_metric': r2_metric})

In [ ]:
predicted = model.predict(X_test)
df_test['predictions'] = predicted

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import numpy as np

predictions = df_test['predictions']
Y_test = df_test['next_day_close']

mse = mean_squared_error(df_test['next_day_close'], df_test['predictions'])
rmse = np.sqrt(mse)
print('Root Mean Squared Error:', rmse)
print("MAPE:", mean_absolute_percentage_error(Y_test, predictions))
print("MAE:", mean_absolute_error(Y_test, predictions))
print("r2:", r2_score(Y_test, predictions))
plt.figure(figsize=(10, 5))  
# Plotting with Matplotlib
plt.plot(df_test.index, df_test.next_day_close, label='Actual')
plt.plot(df_test.index, predictions, color='red', label='Predicted')
plt.title('Нейронная сеть LSTM Предсказания')
plt.xlabel('Дата')
plt.ylabel('Цена')
plt.xticks(rotation=45)
plt.legend()
plt.show()